In [1]:
from monomial_neural_network import *
from experiment import *

In [2]:
d = 2 # input data dimension
teacher_k = [4] # teacher model hidden layer sizes - 3 layers with increasing number of neurons
# teacher_k = [10] # teacher model hidden layer sizes - 1 layer with 10 neurons

teacher_model = generate_teacher_model(d, teacher_k)
print(teacher_model)

n = 1000 # number of data points
# same data dimension d as before

data = generate_data(n, d, teacher_model)

# verify that the data is generated correctly
print(data[0][0], data[1][0], teacher_model.evaluate(data[0][0]))

student_k = [4] # student model hidden layer sizes - 2 layers with increasing number of neurons
student_model = generate_student_model(d, student_k)
print(student_model)



MonomialNeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=4, bias=False)
    (1): Monomial()
    (2): Linear(in_features=4, out_features=1, bias=False)
  )
)
tensor([1.4225, 0.9157]) tensor([6.2432]) tensor([6.2432])
MonomialNeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=4, bias=False)
    (1): Monomial()
    (2): Linear(in_features=4, out_features=1, bias=False)
  )
)


In [3]:
# train the student
student_model, losses = train(
    model = student_model, 
    x_train = data[0], 
    y_train= data[1], 
    num_epochs = 20000, 
    lr = 1e-3
    )

starting training
Epoch [0/20000], Loss: 31.72252
Epoch [100/20000], Loss: 27.09764
Epoch [200/20000], Loss: 22.83154
Epoch [300/20000], Loss: 18.97034
Epoch [400/20000], Loss: 15.59056
Epoch [500/20000], Loss: 12.77433
Epoch [600/20000], Loss: 10.55288
Epoch [700/20000], Loss: 8.80693
Epoch [800/20000], Loss: 7.28398
Epoch [900/20000], Loss: 5.82847
Epoch [1000/20000], Loss: 4.42367
Epoch [1100/20000], Loss: 3.10599
Epoch [1200/20000], Loss: 1.93103
Epoch [1300/20000], Loss: 0.96729
Epoch [1400/20000], Loss: 0.29509
Epoch [1500/20000], Loss: 0.00539
Epoch [1600/20000], Loss: 0.00000
Epoch [1700/20000], Loss: 0.00000
Epoch [1800/20000], Loss: 0.00000
Epoch [1900/20000], Loss: 0.00000
Epoch [2000/20000], Loss: 0.00000
Epoch [2100/20000], Loss: 0.00000
Epoch [2200/20000], Loss: 0.00000
Epoch [2300/20000], Loss: 0.00000
Epoch [2400/20000], Loss: 0.00000
Epoch [2500/20000], Loss: 0.00000
Epoch [2600/20000], Loss: 0.00000
Epoch [2700/20000], Loss: 0.00000
Epoch [2800/20000], Loss: 0.00000
E

In [ ]:
# lets look at the weight matrices
# print(student_model.layers[0].weight)
student_w = student_model.layers[0].weight.detach().numpy()
student_v = student_model.layers[2].weight.detach().numpy()
# print(student_w)
teacher_w = teacher_model.layers[0].weight.detach().numpy()
teacher_v = teacher_model.layers[2].weight.detach().numpy()
# print(student_w)
# print(teacher_w)
# print(np.squeeze(student_v))
A = student_w.T @ np.diag(np.squeeze(student_v)) @ student_w
Astar = teacher_w.T @ np.diag(np.squeeze(teacher_v)) @ teacher_w
print(A)
print(Astar)
eigA, eigvecA = np.linalg.eig(A)
print(eigA)
# print(Astar)
eigAstar, eigvecAstar = np.linalg.eig(Astar)
print(eigAstar)
e = pop_loss(student_model, teacher_model, d=d, N=100000)
# print(y_teach)
# print(y_stud)
# print(torch.norm(y_teach))
print("Population loss = "+str(e))

[[ 2.9341867  -0.50317925]
 [-0.5031793   1.9277991 ]]
[[ 2.9341826 -0.5031786]
 [-0.5031786  1.9278024]]
[3.142606  1.7193797]
[3.1426027 1.7193824]
Population loss = 7.094969e-10


In [5]:
test_vec = np.array([[1,2]]).T
print(test_vec.T@A@test_vec)
# print(test_vec.T@Astar@test_vec)
test_vec_torch = torch.from_numpy(test_vec.T).type(torch.FloatTensor)
print(test_vec_torch)
print(student_model(test_vec_torch))
print(student_model.layers[2].weight)
print(teacher_model.layers[2].weight)
# print(teacher_model(test_vec_torch))

[[8.63266599]]
tensor([[1., 2.]])
tensor([[8.6327]], grad_fn=<MmBackward0>)
Parameter containing:
tensor([[ 1.2476,  1.2093,  1.1667, -0.9397]], requires_grad=True)
Parameter containing:
tensor([[1., 1., 1., 1.]], requires_grad=True)
